In [13]:
%pip install gymnasium renderlab

%pip install opencv-python

%pip install pygame

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [14]:
import gymnasium as gym
import sys
import keyboard
import numpy as np
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [15]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")

observation, info = env.reset()


In [16]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")

num_actions = env.action_space.n
observation, info = env.reset()

def state_action_to_id(state, action):
    return state * num_actions + action

# Monte Carlo algorithm
def monte_carlo(num_episodes):
    # Initialize action-value function and state-action visit counts
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    N = np.zeros((env.observation_space.n, env.action_space.n))
    
    # Loop through episodes
    for episode in range(num_episodes):
        episode_states_actions = []  # Store visited states and actions in the episode
        state = env.reset()
        done = False
        
        # Generate an episode
        while not done:
            action = np.random.randint(num_actions)  # Random action selection
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_states_actions.append((state, action, reward))
            state = next_state
        
        # Update action-value function
        G = 0
        for t in range(len(episode_states_actions) - 1, -1, -1):
            state, action, reward = episode_states_actions[t]
            G += reward
            state_action_id = state_action_to_id(state, action)
            if state_action_id not in [x[0] for x in episode_states_actions[:t]]:
                N[state][action] += 1
                Q[state][action] += (G - Q[state][action]) / N[state][action]
    
    return Q

# Train the agent
num_episodes = 10
Q_values = monte_carlo(num_episodes)

# Evaluate the agent
num_episodes_eval = 100
total_reward = 0
for _ in range(num_episodes_eval):
    state = env.reset()
    done = False
    while not done:
        action = np.argmax(Q_values[state])
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward

average_reward = total_reward / num_episodes_eval

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [17]:
#Close the environment (stop the program)
env.close()